![title](image/2712757.jpg)

# Стартовая страница

python manage.py runserver  
http://127.0.0.1:8000/

![title](image/django.png)

![title](image/Django-Flow.jpg)

# Создадим собственную View

In [ ]:
# notes/views.py¶ 
from django.http import HttpResponse

def index(request):
    return HttpResponse("My notes")

### URL dispatcher

In [ ]:
# Создать файл notes/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index, name='index'),
]

In [ ]:
# mysite/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('', include('notes.urls')), # добавили пути из нашего приложения
    path('admin/', admin.site.urls),
]

# Проверяем

python manage.py runserver  
http://127.0.0.1:8000/

# QuerySet

In [ ]:
notes = Note.objects.all() # получить все заметки

In [ ]:
# notes/views.py

def index(request):
    # Получить все заметки
    notes = Note.objects.all()

    # Получичить список заголовков заметок
    titles = []
    for note in notes:
        titles.append(note.title)

    # формируем строку
    result = ', '.join(titles)

    return HttpResponse("My notes: {}".format(result))

# Template

1. Создадим папку templates в нашем приложении notes
2. В созданной папке templates, создаём папку notes
3. Создаём файл index.html

In [ ]:
# notes/templates/notes/index.html
<h1>Мои заметки:</h1>
<ul>
    {% for note in notes %}
        <li>{{ note.title }}
        <p>{{ note.body }}</p>        
        </li>
    {% endfor %}
</ul>

In [ ]:
# notes/views.py
from django.shortcuts import render
from django.http import HttpResponse

# import models
from .models import Note

# import render for templates
from django.shortcuts import render

def index(request):
    # Получить все заметки:
    notes = Note.objects.all()
    
    # Определить словать для передачи контекста
    context = {
        'notes': notes
    }
    # отрисовываем template с контекстом
    return render(request, 'notes/index.html', context)


## Собственное API

pip install djangorestframework

In [ ]:
INSTALLED_APPS = [
    ...
    'django.contrib.messages',
    'django.contrib.staticfiles',
    
    'rest_framework',
    'notes',
]

Создадим файл в notes/serializers.py

In [ ]:
# notes/serializers.py
from rest_framework import serializers

from .models import Note


class NoteSerializer(serializers.ModelSerializer):

    class Meta:
        model = Note
        fields = ('id', 'title', 'body', 'created_at')

In [ ]:
# notes/views.py

...

from .models import Note
from rest_framework import viewsets
from .serializers import NoteSerializer


class NoteViewSet(viewsets.ModelViewSet):
    queryset = Note.objects.all().order_by('-created_at')
    serializer_class = NoteSerializer

In [ ]:
Создадим файл notes/router.py

# notes/router.py
from rest_framework import routers
from .views import NoteViewSet

# Создаем router
router = routers.DefaultRouter()

# регистрируем наш ViewSet
router.register('notes', NoteViewSet)

api_urls = router.urls

In [ ]:
# notes/urls.py

from django.urls import path, include
from . import views

from .router import api_urls

urlpatterns = [
    path('api/v1/', include(api_urls)),
    path('', views.index, name='index'),
]

# Проверяем:  
python manage.py runserver  
http://localhost:8000/api/v1/notes/  